In [1]:
import keras
from keras.models import load_model
deepsea_model = load_model("/srv/scratch/avanti/ExPecto/resources/deepseabeluga_keras.h5")

Using TensorFlow backend.
W0823 16:38:50.595997 140706238011136 deprecation_wrapper.py:119] From /users/avanti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0823 16:38:50.628729 140706238011136 deprecation_wrapper.py:119] From /users/avanti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0823 16:38:50.681700 140706238011136 deprecation_wrapper.py:119] From /users/avanti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0823 16:38:50.682928 140706238011136 deprecation_wrapper.py:119] From /users/avanti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Pl

In [2]:
deepsea_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1993, 320)         10560     
_________________________________________________________________
activation_1 (Activation)    (None, 1993, 320)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1986, 320)         819520    
_________________________________________________________________
activation_2 (Activation)    (None, 1986, 320)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1986, 320)         0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 496, 320)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 489, 480)          1229280   
__________

In [3]:
from importlib import reload
import seqdataloader
from seqdataloader.batchproducers.coordbased.coordbatchproducers import SimpleCoordsBatchProducer
from seqdataloader.batchproducers.coordbased.coordstovals.fasta import PyfaidxCoordsToVals

positives_coords_batch_producer = SimpleCoordsBatchProducer(
    bed_file="2000bp_around_hg38_new_k562.narrowPeak.gz",
    batch_size=64)
negatives_coords_batch_producer = SimpleCoordsBatchProducer(
    bed_file="nok562_2000bp_around_hg38_merged_universal_neg_representative_peaks.bed.gz",
    batch_size=64)
coords_to_onehot = PyfaidxCoordsToVals(genome_fasta_path="hg38.fasta")

Heads up: coordinates in bed file are assumed to be on the positive strand; if strand in the bed file is improtant to you, please add that feature to SimpleCoordsBatchProducer
Heads up: coordinates in bed file are assumed to be on the positive strand; if strand in the bed file is improtant to you, please add that feature to SimpleCoordsBatchProducer


In [4]:
positives_preds = []
for batch_idx in range(len(positives_coords_batch_producer)):
    if (batch_idx%100==0):
        print("Done",len(positives_preds))
    positives_preds.extend(deepsea_model.predict(
                            coords_to_onehot(positives_coords_batch_producer[batch_idx])))

Done 0
Done 6400
Done 12800
Done 19200
Done 25600
Done 32000
Done 38400
Done 44800
Done 51200
Done 57600
Done 64000
Done 70400
Done 76800
Done 83200
Done 89600
Done 96000
Done 102400
Done 108800
Done 115200
Done 121600
Done 128000
Done 134400
Done 140800
Done 147200
Done 153600
Done 160000
Done 166400
Done 172800
Done 179200
Done 185600
Done 192000
Done 198400
Done 204800


In [ ]:
negatives_preds = []
for batch_idx in range(len(negatives_coords_batch_producer)):
    if (batch_idx%100==0):
        print("Done",len(negatives_preds))
    negatives_preds.extend(deepsea_model.predict(
                            coords_to_onehot(negatives_coords_batch_producer[batch_idx])))

Done 0
Done 6400
Done 12800
Done 19200
Done 25600
Done 32000
Done 38400
Done 44800
Done 51200
Done 57600
Done 64000
Done 70400
Done 76800


In [ ]:
from sklearn.metrics import roc_auc_score
import numpy as np
y_score = list(np.array(positives_preds)[:,61])+list(np.array(negatives_preds)[:,61])
y_true = np.array([1 for x in positives_preds]
                  +[0 for x in negatives_preds])

In [ ]:
from deeplift.dinuc_shuffle import dinuc_shuffle
shuffle_several_times = lambda s: np.array([dinuc_shuffle(s) for i in range(20)])

#This combine_mult_and_diffref function can be used to generate hypothetical
# importance scores for one-hot encoded sequence.
#Hypothetical scores can be thought of as quick estimates of what the
# contribution *would have been* if a different base were present. Hypothetical
# scores are used as input to the importance score clustering algorithm
# TF-MoDISco (https://github.com/kundajelab/tfmodisco)
# Hypothetical importance scores are discussed more in this pull request:
#  https://github.com/kundajelab/deeplift/pull/36
def combine_mult_and_diffref(mult, orig_inp, bg_data):
    to_return = []
    for l in range(len(mult)):
        projected_hypothetical_contribs = np.zeros_like(bg_data[l]).astype("float")
        assert len(orig_inp[l].shape)==2
        #At each position in the input sequence, we iterate over the one-hot encoding
        # possibilities (eg: for genomic sequence, this is ACGT i.e.
        # 1000, 0100, 0010 and 0001) and compute the hypothetical 
        # difference-from-reference in each case. We then multiply the hypothetical
        # differences-from-reference with the multipliers to get the hypothetical contributions.
        #For each of the one-hot encoding possibilities,
        # the hypothetical contributions are then summed across the ACGT axis to estimate
        # the total hypothetical contribution of each position. This per-position hypothetical
        # contribution is then assigned ("projected") onto whichever base was present in the
        # hypothetical sequence.
        #The reason this is a fast estimate of what the importance scores *would* look
        # like if different bases were present in the underlying sequence is that
        # the multipliers are computed once using the original sequence, and are not
        # computed again for each hypothetical sequence.
        for i in range(orig_inp[l].shape[-1]):
            hypothetical_input = np.zeros_like(orig_inp[l]).astype("float")
            hypothetical_input[:,i] = 1.0
            hypothetical_difference_from_reference = (hypothetical_input[None,:,:]-bg_data[l])
            hypothetical_contribs = hypothetical_difference_from_reference*mult[l]
            projected_hypothetical_contribs[:,:,i] = np.sum(hypothetical_contribs,axis=-1) 
        to_return.append(np.mean(projected_hypothetical_contribs,axis=0))
    return to_return

dinuc_shuff_explainer = shap.DeepExplainer(
    #Importance is computed w.r.t. layers[-2] because that corresponds to the logit
    # of the sigmoid; we do this to avoid plateauing of importance scores due to
    # saturation of the sigmoid, as explained in the DeepLIFT paper
    #Task 61 because this is K562 as per
    # https://github.com/FunctionLab/ExPecto/blob/master/resources/deepsea_beluga_2002_features.tsv
    (deepsea_model.input, deepsea_model.layers[-2].output[:,61]),
    shuffle_several_times,
    combine_mult_and_diffref=combine_mult_and_diffref)


In [ ]:
import h5py
f = h5py.File("scores.h5", mode="r+")
f.create_dataset(
    "hyp_imp",
    (len(positives_coords_batch_producer.coords_list), 2000,4),
    dtype="float32")
f.create_dataset(
    "seq",
    (len(positives_coords_batch_producer.coords_list), 2000,4),
    dtype="float32")

num_done = 0
for batch_idx in range(len(positives_coords_batch_producer)):
    if (batch_idx%10==0):
        print("Done",len(positives_deepshap))
    onehot = coords_to_onehot(positives_coords_batch_producer[batch_idx])
    f["seq"][num_done:num_done+len(onehot)] = onehot
    hyp_imp = dinuc_shuff_explainer.shap_values(onehot)
    f["hyp_imp"][num_done:num_done+len(hypimp)] = hyp_imp
    num_done += len(onehot)